In [2]:
studentInfo = pd.read_csv("../data/studentInfo.csv")
studentRegistration = pd.read_csv("../data/studentRegistration.csv")
studentVle = pd.read_csv("../data/studentVle.csv")
studentAssessment = pd.read_csv("../data/studentAssessment.csv")

In [19]:
import pandas as pd

# Load CSVs
studentInfo = pd.read_csv("../data/studentInfo.csv")
studentRegistration = pd.read_csv("../data/studentRegistration.csv")
studentVle = pd.read_csv("../data/studentVle.csv")
studentAssessment = pd.read_csv("../data/studentAssessment.csv")


# 1. studentInfo ← 기준 테이블
merged_df = studentInfo.copy()

# 2. studentRegistration ← 동일 키 기준 병합
merged_df = merged_df.merge(
    studentRegistration[["id_student", "code_module", "code_presentation", "date_registration", "date_unregistration"]],
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)

# 3. studentVle ← sum_click 총합을 먼저 집계 후 병합
vle_grouped = studentVle.groupby(
    ["id_student", "code_module", "code_presentation"]
)["sum_click"].sum().reset_index()

merged_df = merged_df.merge(
    vle_grouped,
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)
merged_df
# # "id_assessment","id_student","date_submitted","is_banked","score"
# # # 4. studentAssessment ← 학습자별 평균 점수 및 제출률 계산
studentAssessment["score"] = pd.to_numeric(studentAssessment["score"], errors="coerce")
studentAssessment["is_banked"] = pd.to_numeric(studentAssessment["is_banked"], errors="coerce")

assess_grouped = studentAssessment.groupby(
    ["id_student"]
).agg({
    "score": "mean",
    "is_banked": "mean"
}).reset_index().rename(columns={
    "score": "avg_score",
    "is_banked": "banked_ratio"
})
assess_grouped
merged_df = merged_df.merge(
    assess_grouped,
    on=["id_student"],
    how="left"
)
merged_df
merged_df.to_csv("../data/merged_dataset.csv", index=False, encoding="utf-8-sig")
# # # 5. 이탈 여부 라벨 생성
# # merged_df["is_dropout"] = merged_df["date_unregistration"].notna().astype(int)

# # # 6. 결측값 처리
# # merged_df["sum_click"] = merged_df["sum_click"].fillna(0)
# # merged_df["avg_score"] = merged_df["avg_score"].fillna(0)
# # merged_df["banked_ratio"] = merged_df["banked_ratio"].fillna(0)


# # 8. 최종 확인
# print(merged_df.head())
